# Mitiq hands-on tutorial

<img src="unitary_fund_logo.png" width=350>

In [1]:
"""Install Mitiq and supported libraries."""
#!pip install mitiq

#!pip install qiskit
#!pip install pyquil

'Install Mitiq and supported libraries.'

In [2]:
"""Test Mitiq installation and view version information."""
import mitiq
mitiq.about()


Mitiq: A Python toolkit for implementing error mitigation on quantum computers
Authored by: Mitiq team, 2020 & later (https://github.com/unitaryfund/mitiq)

Mitiq Version:	0.1.0dev

Cirq Version:	0.9.0.dev
NumPy Version:	1.18.4
SciPy Version:	1.4.1
PyQuil Version:	2.21.0
Qiskit Version:	0.15.1

Python Version:	3.6.8
Platform Info:	Linux (x86_64)
Install Path:	/home/ryan/phd/unitary-fund/team/repos/mitiq/mitiq/mitiq


In [3]:
"""Imports."""
import qiskit
import cirq
import pyquil

from mitiq import zne

# Three steps to ZNE in Mitiq

1. Define a circuit to run.
1. Define a function which executes the circuit and returns an observable of interest.
1. Use `zne.execute_with_zne` with these ingredients. Optionally, pick a noise scaling method and/or inference technique.

# Example using Qiskit and IBMQ

## (1) Define a circuit to run

Below we define a simple circuit in Qiskit. Circuits can be defined in any supported frameworks: Qiskit, pyQuil, or Cirq.

In [4]:
"""Simple circuit to run."""
def circuit(cz_depth: int = 10):
    """Returns a two-qubit circuit with Hadamards and CZ gates."""
    qreg = qiskit.QuantumRegister(2)
    creg1, creg2 = qiskit.ClassicalRegister(1), qiskit.ClassicalRegister(1)
    circuit = qiskit.QuantumCircuit(qreg, creg1, creg2)
    
    for q in qreg:
        circuit.h(q)
        
    for _ in range(cz_depth):
        circuit.cz(*qreg)
    
    for q, c in zip(qreg, [creg1, creg2]):
        circuit.h(q)
        circuit.measure(q, c)

    return circuit

circ = circuit(cz_depth=10)
print(circ)

      ┌───┐                              ┌───┐┌─┐   
q0_0: ┤ H ├─■──■──■──■──■──■──■──■──■──■─┤ H ├┤M├───
      ├───┤ │  │  │  │  │  │  │  │  │  │ ├───┤└╥┘┌─┐
q0_1: ┤ H ├─■──■──■──■──■──■──■──■──■──■─┤ H ├─╫─┤M├
      └───┘                              └───┘ ║ └╥┘
c0: 1/═════════════════════════════════════════╩══╬═
                                               0  ║ 
                                                  ║ 
c1: 1/════════════════════════════════════════════╩═
                                                  0 


## (2) Define an *executor* function

* An **executor** is a function which inputs a quantum circuit and returns an expectation value. 
* Mitiq treats this function as a black box when implementing error mitigation. 
* The function abstracts away backend information so that the interface is the same for different platforms.

In [5]:
"""Load IBMQ credentials."""
provider = qiskit.IBMQ.load_account()

/home/ryan/phd/unitary-fund/team/envs/envfreshqiskit/lib/python3.6/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


In [6]:
"""IBMQ executor."""
def executor(circuit: qiskit.QuantumCircuit, shots: int = 8192) -> float:
    """Executes a circuit and returns an observable of interest."""
    # Execute the circuit
    job = qiskit.execute(
        experiments=circuit,
        backend=provider.get_backend("ibmq_ourense"),
        optimization_level=0,  # Important for unitary folding
        initial_layout={circuit.qregs[0][0]: 1, circuit.qregs[0][1]: 2},
        shots=shots
    )
    
    # Postprocess measurements to return an observable
    counts = job.result().get_counts()
    if counts.get("0 0"):
        expectation_value = counts.get("0 0") / shots
    else:
        expectation_value = 0.0
    print("Expectation value =", round(expectation_value, 3))
    return expectation_value

*Note*: For demonstration purposes, we print out the expectation value each time the executor is called. This gives insight into how Mitiq uses the executor.

## (3) Use `zne.execute_with_zne`

First we run without error mitigation for comparison.

In [7]:
"""Run without error mitigation."""
unmitigated = executor(circ)
print("\nUnmitigated expectation value:", round(unmitigated, 3))

Expectation value = 0.881

Unmitigated expectation value: 0.881


Now we use zero-noise extrapolation.

In [8]:
"""Run with zero-noise extrapolation."""
mitigated = zne.execute_with_zne(
    circ,
    executor,
    scale_noise=zne.scaling.fold_gates_at_random,
    num_to_average=3,
    factory=zne.inference.RichardsonFactory(scale_factors=[1.0, 2.0, 3.0])
)
print("\nMitigated expectation value:", round(mitigated, 3))

Expectation value = 0.874
Expectation value = 0.873
Expectation value = 0.881
Expectation value = 0.724
Expectation value = 0.747
Expectation value = 0.733
Expectation value = 0.603
Expectation value = 0.596
Expectation value = 0.585

Mitigated expectation value: 1.019


* The optional argument `scale_noise` defines the noise scaling technique. The default value is `fold_gates_at_random`.
* The optional argument `num_to_average` is the number of times to call the executor at the current scale factor. The default value is `1`.
* The optional argument `factory` defines the inference technique and scale factors. The default value is `RichardsonFactory(scale_factors=[1.0, 2.0, 3.0]`). 

# Benchmarks in Mitiq whitepaper

# Two-qubit randomized benchmarking circuits

<img src="fig1.png">

# VQE on $H_2$
<img src="fig2.png">

#  More details on Mitiq

## Supported quantum software libraries

Although we wrote our circuit in Qiskit, we could also use pyQuil or Cirq.

In [9]:
"""Defining a Bell state circuit in each supported framework."""
# Qiskit circuit
qreg = qiskit.QuantumRegister(2)
circ = qiskit.QuantumCircuit(qreg)
circ.h(qreg[0])
circ.cx(*qreg)
print("Qiskit circuit:", circ, sep="\n")

# Cirq circuit
cirq_qreg = cirq.LineQubit.range(2)
cirq_circ = cirq.Circuit(cirq.ops.H.on(cirq_qreg[0]), cirq.ops.CNOT.on(*cirq_qreg))
print("\n\nCirq circuit:", cirq_circ, sep="\n")

# pyQuil program
prog = pyquil.Program(pyquil.gates.H(0), pyquil.gates.CNOT(0, 1))
print("\n\npyQuil program:", prog, sep="\n")

Qiskit circuit:
       ┌───┐     
q31_0: ┤ H ├──■──
       └───┘┌─┴─┐
q31_1: ─────┤ X ├
            └───┘


Cirq circuit:
0: ───H───@───
          │
1: ───────X───


pyQuil program:
H 0
CNOT 0 1



## Noise scaling

One technique for noise scaling is unitary folding. This maps gates (or groups of gates $G$) to $G \mapsto G G^\dagger G$. There are several folding functions in Mitiq.

In [10]:
"""Local folding from left."""
folded = zne.scaling.fold_gates_from_left(
    circ, scale_factor=2
)
print("Folded circuit:", folded, sep="\n")

Folded circuit:
     ┌───┐┌───┐┌───┐     
q_0: ┤ H ├┤ H ├┤ H ├──■──
     └───┘└───┘└───┘┌─┴─┐
q_1: ───────────────┤ X ├
                    └───┘


In [11]:
"""Local folding from right."""
folded = zne.scaling.fold_gates_from_right(
    circ, scale_factor=2
)
print("Folded circuit:", folded, sep="\n")

Folded circuit:
     ┌───┐               
q_0: ┤ H ├──■────■────■──
     └───┘┌─┴─┐┌─┴─┐┌─┴─┐
q_1: ─────┤ X ├┤ X ├┤ X ├
          └───┘└───┘└───┘


In [12]:
"""Global folding."""
folded = zne.scaling.fold_global(
    circ, scale_factor=3
)
print("Folded circuit:", folded, sep="\n")

Folded circuit:
     ┌───┐          ┌───┐┌───┐     
q_0: ┤ H ├──■────■──┤ H ├┤ H ├──■──
     └───┘┌─┴─┐┌─┴─┐└───┘└───┘┌─┴─┐
q_1: ─────┤ X ├┤ X ├──────────┤ X ├
          └───┘└───┘          └───┘


The function `zne.scaling.fold_gates_at_random` selects individual gates at random to fold until the scale factor is reached.

### Fold by fidelity

In [13]:
"""Get a circuit to fold."""
circuit = cirq.testing.random_circuit(qubits=4, n_moments=3, op_density=1.0, random_state=2)
print("Original circuit:", circuit, sep="\n")

Original circuit:
0: ───×───────iSwap───
      │       │
1: ───×───T───iSwap───

2: ───Y───S───iSwap───
              │
3: ───H───X───iSwap───


We can set all single qubit fidelities to one as follows.

In [14]:
"""Fold by fidelity."""
folded = zne.scaling.fold_gates_at_random(
    circuit, scale_factor=3, fidelities={"single": 1.0}
)
print("Folded circuit:", folded, sep="\n")

Folded circuit:
0: ───×───×───×──────────────────iSwap───iSwap──────iSwap───
      │   │   │                  │       │          │
1: ───×───×───×───────T──────────iSwap───iSwap^-1───iSwap───

2: ───Y───S───iSwap───iSwap──────iSwap──────────────────────
              │       │          │
3: ───H───X───iSwap───iSwap^-1───iSwap──────────────────────


This prevents any single qubit gates from being folded. See the [docs](https://mitiq.readthedocs.io/en/latest/guide/guide-zne.html#folding-gates-by-fidelity) for full information and options.

## Inference techniques

Some static inference techniques available in Mitiq are shown below. These all can be used as `factory` arguments in `zne.execute_with_zne`.

In [15]:
"""Static inference techniques."""
# Linear extrapolation: y = a x + b
linear_factory = zne.inference.LinearFactory(scale_factors=[1.0, 2.0, 3.0])

# Quadratic extrapolation: y = a x^2 + b x + c
quadratic_factory = zne.inference.PolyFactory(scale_factors=[1.0, 2.0, 3.0], order=2)

# Exponential extrapolation: y = a e^{b x + c}
exp_factory = zne.inference.ExpFactory(scale_factors=[1.0, 2.0, 3.0], asymptote=0.5)

An adaptive inference technique is shown below. This technique inputs the first scale factor and the total number of scale factors to use. The remaining ones are determined adaptively based on previous calls to the executor.

In [16]:
"""Adaptive inference technique."""
ada_exp_factory = zne.inference.AdaExpFactory(scale_factor=2, steps=5)

### Optimal fit parameters and history

When used as arguments in `zne.execute_with_zne`, factories store the optimal parameters of the fit. These can be accessed by the `opt_params` attribute. *Note*: `opt_params` is `None` until the `reduce` method is called (by `zne.execute_with_zne` or otherwise).

# Custom noise scaling or inference techniques

## Custom noise scaling functions

Custom noise scaling functions can be defined for use in Mitiq. The skeleton of custom noise scaling functions must be as follows.

In [17]:
"""Skeleton for a custom noise scaling method."""
from copy import deepcopy

@mitiq.conversions.converter
def my_scaling_method(
    circuit: cirq.Circuit, 
    scale_factor: float, 
    *args, 
    **kwargs
) -> cirq.Circuit:
    """Skeleton for a custom folding function."""
    # Optional but recommended: Make a copy of the input circuit
    folded_circuit = deepcopy(circuit)
    
    # Define the folding method here by modifying operations in the circuit
    # <Act on operations in folded_circuit>
    
    # Return the folded circuit
    return folded_circuit

The `mitiq.conversions.converter` makes it so any supported circuit can be input to the function. The body of the function should still act on a `cirq.Circuit`, however.

## Custom inference techniques

An example custom inference technique is defined below. This fits a quadratic function to the data (scale factors and expectation values), then forces the ZNE value to be in the interval `[-1, 1]`. 

In [18]:
"""Skeleton for a custom inference technique."""
import numpy as np
from mitiq.zne.inference import BatchedFactory

class MyFactory(BatchedFactory):
    def reduce(self) -> float:
        """Skeleton for a custom (static) inference technique."""
        # Get scale factors and expection values
        scale_factors = self.get_scale_factors()
        exp_vals = self.get_expectation_values()
        
        # Define the custom fit here!
        coeffs = np.polyfit(scale_factors, exp_vals, deg=2)
        zne_value = coeffs[-1]
        
        # Return the zero-noise value
        return np.clip(zne_value, -1.0, 1.0)

Note that one can include `self.opt_params = coeffs` in the body of `reduce` to store the optimal parameters.

# Thanks for listening!

Interested in Mitiq?

- Mitiq GitHub: https://github.com/unitaryfund/mitiq
- Mitiq Documentation: https://mitiq.readthedocs.io
- Contact: ryan@unitary.fund